Установка модуля NCBI:

In [1]:
!wget ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh

--2024-10-25 14:16:12--  ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh
           => ‘install-edirect.sh’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.31, 130.14.250.7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /entrez/entrezdirect ... done.
==> SIZE install-edirect.sh ... 4220
==> PASV ... done.    ==> RETR install-edirect.sh ... done.
Length: 4220 (4.1K) (unauthoritative)

install-edirect.sh  100%[===================>]   4.12K  --.-KB/s    in 0s      

2024-10-25 14:16:16 (8.46 MB/s) - ‘install-edirect.sh’ saved [4220]



In [2]:
!chmod 755 install-edirect.sh

In [3]:
!./install-edirect.sh


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



In [4]:
!echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

In [5]:
!./install-edirect.sh


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



In [6]:
!export PATH=${HOME}/edirect:${PATH}

Скачиваем геном близкородственной бактерии T.oleivorans

In [7]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [8]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.1 MB/s eta 0:00:00


In [11]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt

Создаем словарь скаффолдов и добавляем аннотацию к каждой записи

In [12]:
scaffolds = dict()
for record in SeqIO.parse("scaffolds-3.fasta", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  record.annotations['data_file_division'] = 'BCT'

  scaffolds[record.id] = record

Создаем словарь предсказанных генов, добавляем их в соответствующие скаффолды и сохраняем важные аннотации для каждого

In [13]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaffold = desc[1]
  start, end = int(desc[2]), int(desc[3])
  strand = 1 if desc[4] == '+' else -1

  feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [desc[0]]
  feat.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feat)

  genes[desc[0]] = feat

Добавляем функции белков (из MIL-1)

In [14]:
from Bio import SeqIO

genbank_data = SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank")

mil1_proteins = {}
for feature in genbank_data.features:
    if 'protein_id' in feature.qualifiers and 'product' in feature.qualifiers:
        protein_id = feature.qualifiers['protein_id'][0]
        product_name = feature.qualifiers['product'][0]
        mil1_proteins[protein_id] = product_name


In [17]:
import pandas as pd

names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil1hits.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU73181.1_2732,97.319,373,10,0,1,373,174,546,0.000000e+00,751.0
1,1,lcl|HF680312.1_prot_CCU70870.1_421,93.194,191,13,0,1,191,174,364,1.290000e-120,355.0
2,1,lcl|HF680312.1_prot_CCU73188.1_2739,90.957,188,17,0,1,188,42,229,3.290000e-118,344.0
3,1,lcl|HF680312.1_prot_CCU71297.1_848,91.758,182,15,0,1,182,174,355,6.770000e-103,310.0
4,1,lcl|HF680312.1_prot_CCU73191.1_2742,93.333,75,5,0,56,130,1,75,5.070000e-43,141.0


In [18]:
filtered_hits = mil1hits[mil1hits['sseqid'].str.contains("CCU")]

sorted_hits = filtered_hits.sort_values(by='bitscore', ascending=False)

unique_hits = sorted_hits.drop_duplicates(subset='qseqid')

unique_hits.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
3641,1273,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.0,4296.0
2213,811,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.0,4158.0
7328,2613,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.0,3406.0
3868,1350,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.0,3372.0
6552,2335,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.0,3338.0


Добавляем аннотацию функции белка, полученную из MIL-1 к соответствующему гену в текущем геноме

In [21]:
for i, hit in unique_hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil1_proteins[match]]

Добавляем функции белков из SwissProt

In [22]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

--2024-10-25 15:59:13--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 671614860 (641M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 640.50M  18.1MB/s    in 38s     

2024-10-25 15:59:52 (17.1 MB/s) - ‘uniprot_sprot.dat.gz’ saved [671614860/671614860]



In [23]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [24]:
previd = None
swissgenes = dict()
for line in open('SwissProt_names.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

In [25]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

Сохраняем аннторированный геном

In [26]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

83

In [28]:
!tail -n200 GENOME.gbk

ACCESSION   scaffold74_cov851
VERSION     scaffold74_cov851
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     CDS             complement(4..326)
                     /locus_tag="3599"
                     /translation="VNSADLSQGQLTGTDTKDETFEISTASNRVAVNGMNLTSVAAIDA
                     GSSTGDEITTTESVNAELTSTDNELTASGINFSAINNADLSNGALSGSDNADTFELSGA
                     SLLA"
                     /product="predicted heterotrimeric autotransporter subunit
                     A"
ORIGIN
        1 ttagctaata aactcgcacc ggataattca aaggtatcag cgttatcgct accactcagt
       61 gcgccattgc ttaaatcggc attgttaatc gcactgaagt taataccaga ggctgttaat
      121 tcattgtcag tgcttgttag ctccgcattc actgactcgg tcgttgtgat ctcatcgccc
      181 gtgcttgagc ctgcatcaat ggcagcaacc gaagtcaggt tcatgccgtt aacagccact
      241 cgattagatg ctgtgctgat ctcgaaggtt tcatctttgg tgtcggtacc ggttaattga
      301 ccttgagaaa ggtctgcact attcacg
//
LOCUS       scaffold75_cov1115       209 